# Demonstration for the QandA Module

This notebook provides a straightforward guide on how to use the `QandA` object from the `qanda.py` module. The `QandA` object encapsulates a complete Retrieval-Augmented Generation (RAG) pipeline, allowing you to ask questions of a document and receive answers grounded in its content.

## 1. Setup and Initialization

First, we import the necessary libraries and define the configuration parameters for our RAG pipeline. These parameters tell the `QandA` object which models to use, which document to load, and how to structure its prompts.

In [1]:
from pathlib import Path
import warnings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from qanda import QandA

warnings.filterwarnings("ignore", category=UserWarning)

FILE_PATH = Path("jsondata/Rodier-Finding.jsonl")
GEN_MODEL = "gemma3"
EMBED_MODEL = "mxbai-embed-large"
VDB = InMemoryVectorStore
TOP_K = 3
PROMPT = ChatPromptTemplate.from_template(
    """Context information is below.\n
    ---------------------\n
    {context}\n
    ---------------------\n
    Given the context information and not prior knowledge, answer the query.\n
    Query: {input}\n
    Answer:\n"""
)

Now, we can create an instance of the `QandA` object. This will load the models and process the specified document, preparing the pipeline for questions.

In [2]:
qanda = QandA(gen_model=GEN_MODEL,
              embed_model=EMBED_MODEL, 
              vdb=VDB,
              file_path=FILE_PATH,
              top_k=TOP_K,
              prompt=PROMPT)

Initializing, please wait...
Loading jsondata\Rodier-Finding.jsonl
Question Answer chain ready.


## 2. Basic Usage: Asking a Question

The simplest way to use the object is to call the `ask()` method with your question. It will return the answer as a string.

In [3]:
questions = [
    "Who is the coroner?",
    "Who is the deceased?"
]

for question in questions:
    answer = qanda.ask(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: Who is the coroner?
Answer: Sarah Helen Linton, Deputy State Coroner.

Question: Who is the deceased?
Answer: Frank Edward Rodier is the deceased.



## 3. Advanced Usage: Verifying with Sources

A key feature of RAG is transparency. To see the exact text chunks that the LLM used as context to generate an answer, set the `verbose` parameter to `True`. This is crucial for verifying the accuracy and trustworthiness of the response.

The method will return a tuple: `(answer, sources)`.

In [4]:
question = "What was the cause of death?"
answer, sources = qanda.ask(question, verbose=True)

print(f"Question: {question}")
print(f"Answer: {answer}\n")
print("--- Sources Used ---")

for src in sources:
    text_preview = src['text'].replace('\n', ' ')[:200] + '...'
    print(f"Source {src['source']}:")
    print(f"  - Page: {src['page']}")
    print(f"  - Document: {src['document']}")
    print(f"  - Text: {text_preview}")

print("--------------------")

Question: What was the cause of death?
Answer: The cause of death remains unascertained. The coroner determined it was likely an accident, but could not definitively say whether injuries from the rocks contributed to his death.

--- Sources Used ---
Source 1:
  - Page: 6
  - Document: data/Rodier-Finding.pdf
  - Text: IS DEATH ESTABLISHED? 17. As is clear from the above; I am satisfied beyond reasonable doubt that Frank Rodier is deceased and that he died on 25 1975 in the sea after he was washed off the rocks whil...
Source 2:
  - Page: 3
  - Document: data/Rodier-Finding.pdf
  - Text: INTRODUCTION - 2 In my capacity as the Acting State Coroner, I determined on the basis of information provided by the WA Police in August 2023 that   there was   reasonable cause to suspect that Frank...
Source 3:
  - Page: 4
  - Document: data/Rodier-Finding.pdf
  - Text: [2024] WACOR 35 impediment; associated with events at the time of his birth. He had no known medical conditions that would have shor

## 4. Conclusion

This notebook has demonstrated the core functionalities of the `qanda.py` module. By initializing the `QandA` object with the desired configuration, you can easily perform verifiable, source-grounded question answering on local documents.